In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import deque

# --- NOVA FUNÇÃO PARA CALCULAR O ATR ---
def calcular_atr(df, periodo=14):
    """
    Calcula o Average True Range (ATR).
    """
    df['H-L'] = df['Alta'] - df['Baixa']
    df['H-pC'] = abs(df['Alta'] - df['Fechamento'].shift(1))
    df['L-pC'] = abs(df['Baixa'] - df['Fechamento'].shift(1))
    df['TR'] = df[['H-L', 'H-pC', 'L-pC']].max(axis=1)
    df['ATR'] = df['TR'].ewm(span=periodo, adjust=False).mean()
    df.drop(['H-L', 'H-pC', 'L-pC', 'TR'], axis=1, inplace=True)
    return df

# ... (As funções identificar_pivos_robustos e calcular_adx permanecem as mesmas) ...
def calcular_adx(df, periodo=14):
    df['H-L'] = df['Alta'] - df['Baixa']; df['H-pC'] = abs(df['Alta'] - df['Fechamento'].shift(1)); df['L-pC'] = abs(df['Baixa'] - df['Fechamento'].shift(1))
    df['TR'] = df[['H-L', 'H-pC', 'L-pC']].max(axis=1)
    df['+DM'] = np.where((df['Alta'] - df['Alta'].shift(1)) > (df['Baixa'].shift(1) - df['Baixa']), df['Alta'] - df['Alta'].shift(1), 0)
    df['+DM'] = np.where(df['+DM'] < 0, 0, df['+DM'])
    df['-DM'] = np.where((df['Baixa'].shift(1) - df['Baixa']) > (df['Alta'] - df['Alta'].shift(1)), df['Baixa'].shift(1) - df['Baixa'], 0)
    df['-DM'] = np.where(df['-DM'] < 0, 0, df['-DM'])
    ATR = df['TR'].ewm(alpha=1/periodo, adjust=False).mean()
    df['+DI'] = (df['+DM'].ewm(alpha=1/periodo, adjust=False).mean() / ATR) * 100
    df['-DI'] = (df['-DM'].ewm(alpha=1/periodo, adjust=False).mean() / ATR) * 100
    dx = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = dx.ewm(alpha=1/periodo, adjust=False).mean()
    df.drop(['H-L', 'H-pC', 'L-pC', 'TR', '+DM', '-DM'], axis=1, inplace=True)
    return df

def identificar_pivos_robustos(df, periodo_verificacao=5):
    topos = []; fundos = []
    janela_alta = deque(maxlen=2 * periodo_verificacao + 1); janela_baixa = deque(maxlen=2 * periodo_verificacao + 1); indices_janela = deque(maxlen=2 * periodo_verificacao + 1)
    for i in range(len(df)):
        janela_alta.append(df['Alta'].iloc[i]); janela_baixa.append(df['Baixa'].iloc[i]); indices_janela.append(i)
        if len(janela_alta) == 2 * periodo_verificacao + 1:
            indice_central = indices_janela[periodo_verificacao]; preco_central_alta = janela_alta[periodo_verificacao]; preco_central_baixa = janela_baixa[periodo_verificacao]
            if preco_central_alta == max(janela_alta): topos.append((indice_central, preco_central_alta))
            if preco_central_baixa == min(janela_baixa): fundos.append((indice_central, preco_central_baixa))
    pivos_alta_confirmados = []
    if len(fundos) >= 2 and len(topos) >= 1:
        for i in range(len(topos)):
            for j in range(len(fundos) - 1):
                p1_idx, p1_preco = fundos[j]; p2_idx, p2_preco = topos[i]; p3_idx, p3_preco = fundos[j+1]
                if (p1_idx < p2_idx < p3_idx) and (p3_preco > p1_preco):
                    for k in range(p3_idx + 1, len(df)):
                        if df['Alta'].iloc[k] > p2_preco:
                            pivos_alta_confirmados.append({'ponto1': p1_preco, 'ponto2': p2_preco, 'ponto3': p3_preco, 'indice_rompimento': k, 'data_p1': df.index[p1_idx], 'data_p2': df.index[p2_idx], 'data_p3': df.index[p3_idx]}); break
    pivos_baixa_confirmados = []
    if len(topos) >= 2 and len(fundos) >= 1:
        for i in range(len(fundos)):
            for j in range(len(topos) - 1):
                p1_idx, p1_preco = topos[j]; p2_idx, p2_preco = fundos[i]; p3_idx, p3_preco = topos[j+1]
                if (p1_idx < p2_idx < p3_idx) and (p3_preco < p1_preco):
                    for k in range(p3_idx + 1, len(df)):
                        if df['Baixa'].iloc[k] < p2_preco:
                            pivos_baixa_confirmados.append({'ponto1': p1_preco, 'ponto2': p2_preco, 'ponto3': p3_preco, 'indice_rompimento': k, 'data_p1': df.index[p1_idx], 'data_p2': df.index[p2_idx], 'data_p3': df.index[p3_idx]}); break
    pivos_alta_final = []; seen_alta = set()
    for pivo in pivos_alta_confirmados:
        identifier = (pivo['ponto1'], pivo['ponto2'], pivo['ponto3'], pivo['indice_rompimento'])
        if identifier not in seen_alta: seen_alta.add(identifier); pivos_alta_final.append(pivo)
    pivos_baixa_final = []; seen_baixa = set()
    for pivo in pivos_baixa_confirmados:
        identifier = (pivo['ponto1'], pivo['ponto2'], pivo['ponto3'], pivo['indice_rompimento'])
        if identifier not in seen_baixa: seen_baixa.add(identifier); pivos_baixa_final.append(pivo)
    pivos_alta_final.sort(key=lambda x: x['indice_rompimento']); pivos_baixa_final.sort(key=lambda x: x['indice_rompimento'])
    return pivos_alta_final, pivos_baixa_final

# --- FUNÇÃO DE BACKTEST FINAL COM GERENCIAMENTO DE RISCO ---
def executar_backtest_final(df, lista_pivos, tipo_pivo, adx_minimo=25, 
                           multiplicador_atr=2.0, relacao_risco_retorno=1.5):
    resultados_trades = []
    for id_pivo, pivo in enumerate(lista_pivos, 1):
        indice_rompimento = pivo['indice_rompimento']
        indice_entrada = indice_rompimento + 1
        if indice_entrada >= len(df): continue

        # Filtro triplo de qualidade
        ema_rapida = df['ema_rapida'].iloc[indice_entrada]; ema_lenta = df['ema_lenta'].iloc[indice_entrada]
        adx = df['ADX'].iloc[indice_entrada]; di_pos = df['+DI'].iloc[indice_entrada]; di_neg = df['-DI'].iloc[indice_entrada]
        
        condicao_valida = False
        if tipo_pivo == 'alta' and ema_rapida > ema_lenta and adx > adx_minimo and di_pos > di_neg:
            condicao_valida = True
        elif tipo_pivo == 'baixa' and ema_rapida < ema_lenta and adx > adx_minimo and di_neg > di_pos:
            condicao_valida = True

        if not condicao_valida: continue

        # --- NOVO GERENCIAMENTO DE RISCO E ALVO ---
        preco_entrada = df['Abertura'].iloc[indice_entrada]
        atr_no_ponto = df['ATR'].iloc[indice_entrada]
        
        # Define Stop e Alvo com base no ATR e na Relação Risco/Retorno
        distancia_stop = multiplicador_atr * atr_no_ponto
        distancia_alvo = distancia_stop * relacao_risco_retorno

        if tipo_pivo == 'alta':
            preco_stop = preco_entrada - distancia_stop
            preco_alvo = preco_entrada + distancia_alvo
        else: # tipo_pivo == 'baixa'
            preco_stop = preco_entrada + distancia_stop
            preco_alvo = preco_entrada - distancia_alvo
        # --- FIM DO NOVO GERENCIAMENTO ---

        trade_info = {"ID Pivô": id_pivo, "Tipo": "COMPRA" if tipo_pivo == 'alta' else "VENDA", "Data Entrada": df.index[indice_entrada].strftime('%Y-%m-%d'), "Preço Entrada": preco_entrada, "Preço Alvo": preco_alvo, "Preço Stop": preco_stop, "Resultado": "INDETERMINADO", "P/L (R$)": 0.0}
        
        for i in range(indice_entrada, len(df)):
            candle_high, candle_low = df['Alta'].iloc[i], df['Baixa'].iloc[i]
            atingiu_stop, atingiu_alvo = False, False
            if tipo_pivo == 'alta':
                if candle_low <= preco_stop: atingiu_stop = True
                if candle_high >= preco_alvo: atingiu_alvo = True
            else:
                if candle_high >= preco_stop: atingiu_stop = True
                if candle_low <= preco_alvo: atingiu_alvo = True
            if atingiu_alvo and not atingiu_stop:
                trade_info["Resultado"] = "LUCRO"; lucro = (preco_alvo - preco_entrada if tipo_pivo == 'alta' else preco_entrada - preco_alvo) * 0.20; trade_info["P/L (R$)"] = lucro; break
            if atingiu_stop:
                trade_info["Resultado"] = "PREJUÍZO"; prejuizo = (preco_stop - preco_entrada if tipo_pivo == 'alta' else preco_entrada - preco_stop) * 0.20; trade_info["P/L (R$)"] = prejuizo; break
        resultados_trades.append(trade_info)
    return resultados_trades

def imprimir_resultados_backtest(titulo, resultados):
    if not resultados:
        print(f"\n--- {titulo} ---"); print("Nenhum trade realizado."); return 0.0
    print(f"\n--- {titulo} ---")
    header = "| {:<7} | {:<7} | {:<12} | {:<13} | {:<12} | {:<12} | {:<10} | {:<10} |".format("ID Pivô", "Tipo", "Data Ent.", "Preço Ent.", "Preço Alvo", "Preço Stop", "Resultado", "P/L (R$)")
    print(header); print("-" * len(header))
    lucro_total = 0; lucros = 0; prejuizos = 0
    for trade in resultados:
        lucro_total += trade["P/L (R$)"]
        if trade["Resultado"] == "LUCRO": lucros += 1
        elif trade["Resultado"] == "PREJUÍZO": prejuizos += 1
        print("| {:<7} | {:<7} | {:<12} | {:<13.2f} | {:<12.2f} | {:<12.2f} | {:<10} | R${:<9.2f} |".format(trade["ID Pivô"], trade["Tipo"], trade["Data Entrada"], trade["Preço Entrada"], trade["Preço Alvo"], trade["Preço Stop"], trade["Resultado"], trade["P/L (R$)"]))
    print("-" * len(header)); print("\nResumo:")
    total_trades = len(resultados)
    win_rate = (lucros / total_trades * 100) if total_trades > 0 else 0
    print(f"  - Trades Totais: {total_trades}\n  - Trades com Lucro: {lucros}\n  - Trades com Prejuízo: {prejuizos}")
    print(f"  - Taxa de Acerto: {win_rate:.2f}%")
    print(f"  - Lucro/Prejuízo Total: R$ {lucro_total:.2f}")
    return lucro_total


# --- SCRIPT PRINCIPAL ---

if __name__ == "__main__":
    try:
        df = pd.read_excel(r"c:\Users\danie\OneDrive\Área de Trabalho\Todas as pastas\Treinamento Modelo\renko_teste.xlsm", sheet_name="Destino")
        df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
        df = df.loc[~df.index.duplicated(keep='first')]; df = df.set_index('Data')
    except FileNotFoundError:
        print("Arquivo não encontrado. Usando dados de exemplo para demonstração.\n")
        datas = pd.to_datetime(pd.date_range(start='2024-01-01', periods=300))
        precos = 100 + np.sin(np.linspace(0, 40, 300)) * 15 + np.random.randn(300).cumsum() * 0.4
        dados = {'Data': datas, 'Abertura': precos, 'Alta': precos + np.random.uniform(0.5, 2, 300), 'Baixa': precos - np.random.uniform(0.5, 2, 300), 'Fechamento': precos + np.random.uniform(-0.5, 0.5, 300)}
        df = pd.DataFrame(dados).set_index('Data')
    
    # --- CÁLCULO DOS INDICADORES ---
    df['ema_rapida'] = df['Fechamento'].ewm(span=21, adjust=False).mean()
    df['ema_lenta'] = df['Fechamento'].ewm(span=50, adjust=False).mean()
    df = calcular_adx(df, periodo=14)
    df = calcular_atr(df, periodo=14) # Calcula o ATR para o gerenciamento de risco
    df.dropna(inplace=True) # Remove linhas com NaN geradas pelos cálculos iniciais
    
    # --- DETECÇÃO E BACKTEST FINAL ---
    lista_pivos_alta, lista_pivos_baixa = identificar_pivos_robustos(df, periodo_verificacao=5)
    print(f"\n[INFO] Encontrado(s) {len(lista_pivos_alta)} pivô(s) de alta brutos.")
    print(f"[INFO] Encontrado(s) {len(lista_pivos_baixa)} pivô(s) de baixa brutos.")

    # Parâmetros de Risco - SINTA-SE À VONTADE PARA TESTAR OUTROS VALORES
    MULT_ATR_STOP = 2.0       # Distância do Stop = 2x o ATR
    RELACAO_RR = 1.5          # Relação Risco/Retorno = 1 para 1.5

    resultados_alta = executar_backtest_final(df, lista_pivos_alta, 'alta', adx_minimo=25, multiplicador_atr=MULT_ATR_STOP, relacao_risco_retorno=RELACAO_RR)
    lucro_total_alta = imprimir_resultados_backtest("Backtest de COMPRAS (Risco/Retorno Fixo)", resultados_alta)

    resultados_baixa = executar_backtest_final(df, lista_pivos_baixa, 'baixa', adx_minimo=25, multiplicador_atr=MULT_ATR_STOP, relacao_risco_retorno=RELACAO_RR)
    lucro_total_baixa = imprimir_resultados_backtest("Backtest de VENDAS (Risco/Retorno Fixo)", resultados_baixa)

    # --- Resumo Final ---
    lucro_geral = lucro_total_alta + lucro_total_baixa
    print("\n" + "="*40); print("        RESUMO GERAL DA ESTRATÉGIA      "); print("="*40)
    print(f"Lucro/Prejuízo Total (Compras): R$ {lucro_total_alta:.2f}")
    print(f"Lucro/Prejuízo Total (Vendas):  R$ {lucro_total_baixa:.2f}")
    print("-"*40)
    print(f"RESULTADO LÍQUIDO TOTAL:        R$ {lucro_geral:.2f}")
    print("="*40)


[INFO] Encontrado(s) 567 pivô(s) de alta brutos.
[INFO] Encontrado(s) 666 pivô(s) de baixa brutos.

--- Backtest de COMPRAS (Risco/Retorno Fixo) ---
| ID Pivô | Tipo    | Data Ent.    | Preço Ent.    | Preço Alvo   | Preço Stop   | Resultado  | P/L (R$)   |
------------------------------------------------------------------------------------------------------------
| 1       | COMPRA  | 2024-08-19   | 141448.71     | 141747.87    | 141249.27    | LUCRO      | R$59.83     |
| 3       | COMPRA  | 2024-08-19   | 142848.73     | 143089.02    | 142688.54    | PREJUÍZO   | R$-32.04    |
| 5       | COMPRA  | 2024-08-20   | 142709.24     | 142989.83    | 142522.18    | LUCRO      | R$56.12     |
| 6       | COMPRA  | 2024-08-20   | 142972.72     | 143226.57    | 142803.49    | PREJUÍZO   | R$-33.85    |
| 17      | COMPRA  | 2024-08-26   | 142905.56     | 143258.86    | 142670.03    | PREJUÍZO   | R$-47.11    |
| 21      | COMPRA  | 2024-08-26   | 143499.66     | 143740.12    | 143339.35    |

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
import itertools

# --- FUNÇÕES DE CÁLCULO, DETECÇÃO E BACKTEST (Sem alterações) ---
def calcular_atr(df, periodo=14):
    df_atr = df.copy(); df_atr['H-L'] = df_atr['Alta'] - df_atr['Baixa']; df_atr['H-pC'] = abs(df_atr['Alta'] - df_atr['Fechamento'].shift(1)); df_atr['L-pC'] = abs(df_atr['Baixa'] - df_atr['Fechamento'].shift(1)); df_atr['TR'] = df_atr[['H-L', 'H-pC', 'L-pC']].max(axis=1); df_atr['ATR'] = df_atr['TR'].ewm(span=periodo, adjust=False).mean(); return df_atr[['ATR']]
def calcular_adx(df, periodo=14):
    df_adx = df.copy(); df_adx['H-L'] = df_adx['Alta'] - df_adx['Baixa']; df_adx['H-pC'] = abs(df_adx['Alta'] - df_adx['Fechamento'].shift(1)); df_adx['L-pC'] = abs(df_adx['Baixa'] - df_adx['Fechamento'].shift(1)); df_adx['TR'] = df_adx[['H-L', 'H-pC', 'L-pC']].max(axis=1); df_adx['+DM'] = np.where((df_adx['Alta'] - df_adx['Alta'].shift(1)) > (df_adx['Baixa'].shift(1) - df_adx['Baixa']), df_adx['Alta'] - df_adx['Alta'].shift(1), 0); df_adx['+DM'] = np.where(df_adx['+DM'] < 0, 0, df_adx['+DM']); df_adx['-DM'] = np.where((df_adx['Baixa'].shift(1) - df_adx['Baixa']) > (df_adx['Alta'] - df_adx['Alta'].shift(1)), df_adx['Baixa'].shift(1) - df_adx['Baixa'], 0); df_adx['-DM'] = np.where(df_adx['-DM'] < 0, 0, df_adx['-DM']); ATR = df_adx['TR'].ewm(alpha=1/periodo, adjust=False).mean(); df_adx['+DI'] = (df_adx['+DM'].ewm(alpha=1/periodo, adjust=False).mean() / ATR) * 100; df_adx['-DI'] = (df_adx['-DM'].ewm(alpha=1/periodo, adjust=False).mean() / ATR) * 100; dx = (abs(df_adx['+DI'] - df_adx['-DI']) / (df_adx['+DI'] + df_adx['-DI'])).fillna(0) * 100; df_adx['ADX'] = dx.ewm(alpha=1/periodo, adjust=False).mean(); return df_adx[['+DI', '-DI', 'ADX']]
def identificar_pivos_robustos(df, periodo_verificacao=5):
    topos = []; fundos = []
    janela_alta = deque(maxlen=2 * periodo_verificacao + 1); janela_baixa = deque(maxlen=2 * periodo_verificacao + 1); indices_janela = deque(maxlen=2 * periodo_verificacao + 1)
    for i in range(len(df)):
        janela_alta.append(df['Alta'].iloc[i]); janela_baixa.append(df['Baixa'].iloc[i]); indices_janela.append(i)
        if len(janela_alta) == 2 * periodo_verificacao + 1:
            indice_central = indices_janela[periodo_verificacao]; preco_central_alta = janela_alta[periodo_verificacao]; preco_central_baixa = janela_baixa[periodo_verificacao]
            if preco_central_alta == max(janela_alta): topos.append((indice_central, preco_central_alta))
            if preco_central_baixa == min(janela_baixa): fundos.append((indice_central, preco_central_baixa))
    pivos_alta_confirmados = []
    if len(fundos) >= 2 and len(topos) >= 1:
        for i in range(len(topos)):
            for j in range(len(fundos) - 1):
                p1_idx, p1_preco = fundos[j]; p2_idx, p2_preco = topos[i]; p3_idx, p3_preco = fundos[j+1]
                if (p1_idx < p2_idx < p3_idx) and (p3_preco > p1_preco):
                    for k in range(p3_idx + 1, len(df)):
                        if df['Alta'].iloc[k] > p2_preco:
                            pivos_alta_confirmados.append({'ponto1': p1_preco, 'ponto2': p2_preco, 'ponto3': p3_preco, 'indice_rompimento': k, 'data_p1': df.index[p1_idx], 'data_p2': df.index[p2_idx], 'data_p3': df.index[p3_idx]}); break
    pivos_baixa_confirmados = []
    if len(topos) >= 2 and len(fundos) >= 1:
        for i in range(len(fundos)):
            for j in range(len(topos) - 1):
                p1_idx, p1_preco = topos[j]; p2_idx, p2_preco = fundos[i]; p3_idx, p3_preco = topos[j+1]
                if (p1_idx < p2_idx < p3_idx) and (p3_preco < p1_preco):
                    for k in range(p3_idx + 1, len(df)):
                        if df['Baixa'].iloc[k] < p2_preco:
                            pivos_baixa_confirmados.append({'ponto1': p1_preco, 'ponto2': p2_preco, 'ponto3': p3_preco, 'indice_rompimento': k, 'data_p1': df.index[p1_idx], 'data_p2': df.index[p2_idx], 'data_p3': df.index[p3_idx]}); break
    pivos_alta_final = []; seen_alta = set()
    for pivo in pivos_alta_confirmados:
        identifier = (pivo['ponto1'], pivo['ponto2'], pivo['ponto3'], pivo['indice_rompimento'])
        if identifier not in seen_alta: seen_alta.add(identifier); pivos_alta_final.append(pivo)
    pivos_baixa_final = []; seen_baixa = set()
    for pivo in pivos_baixa_confirmados:
        identifier = (pivo['ponto1'], pivo['ponto2'], pivo['ponto3'], pivo['indice_rompimento'])
        if identifier not in seen_baixa: seen_baixa.add(identifier); pivos_baixa_final.append(pivo)
    pivos_alta_final.sort(key=lambda x: x['indice_rompimento']); pivos_baixa_final.sort(key=lambda x: x['indice_rompimento'])
    return pivos_alta_final, pivos_baixa_final

def executar_backtest_final(df, lista_pivos, tipo_pivo, adx_minimo=25, multiplicador_atr=2.0, relacao_risco_retorno=1.5):
    resultados_trades = []; 
    for id_pivo, pivo in enumerate(lista_pivos, 1):
        indice_rompimento = pivo['indice_rompimento']; indice_entrada = indice_rompimento + 1
        if indice_entrada >= len(df): continue
        ema_rapida = df['ema_rapida'].iloc[indice_entrada]; ema_lenta = df['ema_lenta'].iloc[indice_entrada]
        adx = df['ADX'].iloc[indice_entrada]; di_pos = df['+DI'].iloc[indice_entrada]; di_neg = df['-DI'].iloc[indice_entrada]
        condicao_valida = False
        if tipo_pivo == 'alta' and ema_rapida > ema_lenta and adx > adx_minimo and di_pos > di_neg: condicao_valida = True
        elif tipo_pivo == 'baixa' and ema_rapida < ema_lenta and adx > adx_minimo and di_neg > di_pos: condicao_valida = True
        if not condicao_valida: continue
        preco_entrada = df['Abertura'].iloc[indice_entrada]; atr_no_ponto = df['ATR'].iloc[indice_entrada]
        distancia_stop = multiplicador_atr * atr_no_ponto; distancia_alvo = distancia_stop * relacao_risco_retorno
        if tipo_pivo == 'alta': preco_stop = preco_entrada - distancia_stop; preco_alvo = preco_entrada + distancia_alvo
        else: preco_stop = preco_entrada + distancia_stop; preco_alvo = preco_entrada - distancia_alvo
        trade_info = {"ID Pivô": id_pivo, "Tipo": "COMPRA" if tipo_pivo == 'alta' else "VENDA", "Data Entrada": df.index[indice_entrada], "Preço Entrada": preco_entrada, "Preço Alvo": preco_alvo, "Preço Stop": preco_stop, "Resultado": "INDETERMINADO", "P/L (R$)": 0.0}
        for i in range(indice_entrada, len(df)):
            candle_high, candle_low = df['Alta'].iloc[i], df['Baixa'].iloc[i]
            atingiu_stop, atingiu_alvo = False, False
            if tipo_pivo == 'alta':
                if candle_low <= preco_stop: atingiu_stop = True
                if candle_high >= preco_alvo: atingiu_alvo = True
            else:
                if candle_high >= preco_stop: atingiu_stop = True
                if candle_low <= preco_alvo: atingiu_alvo = True
            if atingiu_alvo and not atingiu_stop:
                trade_info["Resultado"] = "LUCRO"; lucro = abs(preco_alvo - preco_entrada) * 0.20; trade_info["P/L (R$)"] = lucro; break
            if atingiu_stop:
                trade_info["Resultado"] = "PREJUÍZO"; prejuizo = -abs(preco_stop - preco_entrada) * 0.20; trade_info["P/L (R$)"] = prejuizo; break
        resultados_trades.append(trade_info)
    return resultados_trades

def imprimir_resultados_backtest(titulo, resultados, verbose=True):
    if not verbose: return sum(trade["P/L (R$)"] for trade in resultados)
    if not resultados: print(f"\n--- {titulo} ---"); print("Nenhum trade realizado."); return 0.0
    print(f"\n--- {titulo} ---")
    header = "| {:<7} | {:<7} | {:<12} | {:<13} | {:<12} | {:<12} | {:<10} | {:<10} |".format("ID Pivô", "Tipo", "Data Ent.", "Preço Ent.", "Preço Alvo", "Preço Stop", "Resultado", "P/L (R$)")
    print(header); print("-" * len(header))
    lucro_total = 0; lucros = 0; prejuizos = 0
    for trade in resultados:
        lucro_total += trade["P/L (R$)"]
        if trade["Resultado"] == "LUCRO": lucros += 1
        elif trade["Resultado"] == "PREJUÍZO": prejuizos += 1
        print("| {:<7} | {:<7} | {:<12} | {:<13.2f} | {:<12.2f} | {:<12.2f} | {:<10} | R${:<9.2f} |".format(trade["ID Pivô"], trade["Tipo"], trade["Data Entrada"].strftime('%Y-%m-%d'), trade["Preço Entrada"], trade["Preço Alvo"], trade["Preço Stop"], trade["Resultado"], trade["P/L (R$)"]))
    print("-" * len(header)); print("\nResumo:")
    total_trades = len(resultados)
    win_rate = (lucros / total_trades * 100) if total_trades > 0 else 0
    print(f"  - Trades Totais: {total_trades}\n  - Trades com Lucro: {lucros}\n  - Trades com Prejuízo: {prejuizos}")
    print(f"  - Taxa de Acerto: {win_rate:.2f}%")
    print(f"  - Lucro/Prejuízo Total: R$ {lucro_total:.2f}")
    return lucro_total


if __name__ == "__main__":
    try:
        df_completo = pd.read_excel(r"c:\Users\danie\OneDrive\Área de Trabalho\Todas as pastas\Treinamento Modelo\renko_teste.xlsm", sheet_name="Destino")
        df_completo['Data'] = pd.to_datetime(df_completo['Data'], dayfirst=True)
        df_completo = df_completo.loc[~df_completo.index.duplicated(keep='first')]
        df_completo = df_completo.set_index('Data')
    except FileNotFoundError:
        print("Arquivo não encontrado. Usando dados de exemplo para demonstração.\n")
        datas = pd.to_datetime(pd.date_range(start='2022-01-01', periods=1000))
        precos = 100 + np.sin(np.linspace(0, 80, 1000)) * 25 + np.random.randn(1000).cumsum() * 0.6
        df_completo = pd.DataFrame(index=datas)
        df_completo['Abertura'] = precos; df_completo['Fechamento'] = precos; df_completo['Alta'] = precos; df_completo['Baixa'] = precos

    ponto_divisao = int(len(df_completo) * 0.70)
    df_in_sample = df_completo.iloc[:ponto_divisao].copy()
    df_out_of_sample = df_completo.iloc[ponto_divisao:].copy()
    print(f"Dados divididos em {len(df_in_sample)} registros para Otimização (In-Sample) e {len(df_out_of_sample)} para Validação (Out-of-Sample).")
    
    print("\n" + "="*50); print("        INICIANDO ETAPA 1: OTIMIZAÇÃO IN-SAMPLE        "); print("="*50)
    
    df_is = df_in_sample
    df_is['ema_rapida'] = df_is['Fechamento'].ewm(span=21, adjust=False).mean()
    df_is['ema_lenta'] = df_is['Fechamento'].ewm(span=50, adjust=False).mean()
    adx_df_is = calcular_adx(df_is.copy(), periodo=14)
    atr_df_is = calcular_atr(df_is.copy(), periodo=14)
    df_is = df_is.join([adx_df_is, atr_df_is])
    df_is.dropna(inplace=True)
    
    lista_pivos_alta_is, lista_pivos_baixa_is = identificar_pivos_robustos(df_is, periodo_verificacao=5)
    
    relacao_rr_range = [1.5, 2.0]
    mult_atr_stop_range = [1.5, 2.0, 2.5]
    adx_minimo_range = [22, 25]
    combinacoes = list(itertools.product(relacao_rr_range, mult_atr_stop_range, adx_minimo_range))
    
    melhor_lucro_is = -float('inf'); melhores_parametros = None
    print(f"Iniciando otimização In-Sample com {len(combinacoes)} combinações...")
    for i, (rr, atr_mult, adx_min) in enumerate(combinacoes):
        res_alta = executar_backtest_final(df_is, lista_pivos_alta_is, 'alta', adx_min, atr_mult, rr)
        res_baixa = executar_backtest_final(df_is, lista_pivos_baixa_is, 'baixa', adx_min, atr_mult, rr)
        lucro_geral = sum(t["P/L (R$)"] for t in res_alta) + sum(t["P/L (R$)"] for t in res_baixa)
        if lucro_geral > melhor_lucro_is:
            melhor_lucro_is = lucro_geral
            # --- LINHA CORRIGIDA ---
            # As chaves do dicionário agora correspondem aos nomes exatos dos parâmetros da função
            melhores_parametros = {
                'relacao_risco_retorno': rr, 
                'multiplicador_atr': atr_mult, 
                'adx_minimo': adx_min
            }
            # --- FIM DA CORREÇÃO ---
    
    print("\n--- OTIMIZAÇÃO IN-SAMPLE FINALIZADA ---")
    print("Melhores Parâmetros Encontrados:")
    for param, valor in melhores_parametros.items(): print(f"  - {param}: {valor}")
    print(f"Melhor Lucro Líquido no período In-Sample: R$ {melhor_lucro_is:.2f}")

    print("\n" + "="*50); print("        INICIANDO ETAPA 2: VALIDAÇÃO OUT-OF-SAMPLE        "); print("="*50)
    print("Aplicando os melhores parâmetros encontrados em dados novos (não vistos)...")

    df_oos = df_out_of_sample
    df_oos['ema_rapida'] = df_oos['Fechamento'].ewm(span=21, adjust=False).mean()
    df_oos['ema_lenta'] = df_oos['Fechamento'].ewm(span=50, adjust=False).mean()
    adx_df_oos = calcular_adx(df_oos.copy(), periodo=14)
    atr_df_oos = calcular_atr(df_oos.copy(), periodo=14)
    df_oos = df_oos.join([adx_df_oos, atr_df_oos])
    df_oos.dropna(inplace=True)
    
    lista_pivos_alta_oos, lista_pivos_baixa_oos = identificar_pivos_robustos(df_oos, periodo_verificacao=5)

    resultados_alta_oos = executar_backtest_final(df_oos, lista_pivos_alta_oos, 'alta', **melhores_parametros)
    lucro_total_alta_oos = imprimir_resultados_backtest("Backtest de COMPRAS (Out-of-Sample)", resultados_alta_oos)

    resultados_baixa_oos = executar_backtest_final(df_oos, lista_pivos_baixa_oos, 'baixa', **melhores_parametros)
    lucro_total_baixa_oos = imprimir_resultados_backtest("Backtest de VENDAS (Out-of-Sample)", resultados_baixa_oos)

    lucro_geral_final_oos = lucro_total_alta_oos + lucro_total_baixa_oos
    print("\n" + "="*40); print("        RESULTADO FINAL DA VALIDAÇÃO (Out-of-Sample)      "); print("="*40)
    print(f"Lucro/Prejuízo Total (Compras): R$ {lucro_total_alta_oos:.2f}")
    print(f"Lucro/Prejuízo Total (Vendas):  R$ {lucro_total_baixa_oos:.2f}")
    print("-"*40)
    print(f"RESULTADO LÍQUIDO NA GENERALIZAÇÃO:  R$ {lucro_geral_final_oos:.2f}")
    print("="*40)

Dados divididos em 13645 registros para Otimização (In-Sample) e 5848 para Validação (Out-of-Sample).

        INICIANDO ETAPA 1: OTIMIZAÇÃO IN-SAMPLE        
